In [6]:
from datetime import datetime
from dateutil.relativedelta import relativedelta
import calendar
import json


In [8]:
item_list = [
    {
        "idx": 1,
        "item_code": "Executive Desk (4*2)",
        "sales_description": "Dedicated Executive Desk",
        "qty": 10,
        "rate": "1000",
        "amount": "10000",
        "start_date": "2023-11-01",
        "stop_date": "2024-10-17",
    },
    {
        "idx": 2,
        "item_code": "Executive Desk (4*2)",
        "qty": "10",
        "rate": "1080",
        "amount": "10800",
        "start_date": "2024-10-18",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 3,
        "item_code": "Executive Desk (4*2)",
        "qty": 15,
        "rate": "1080",
        "amount": "16200",
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 4,
        "item_code": "Executive Desk (4*2)",
        "qty": 5,
        "rate": "1000",
        "amount": "5000",
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 5,
        "item_code": "Manager Cabin",
        "qty": 5,
        "rate": 5000,
        "amount": 25000,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 6,
        "item_code": "Manager Cabin",
        "qty": 7,
        "rate": "5000",
        "amount": 35000,
        "start_date": "2024-12-15",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 7,
        "item_code": "Manager Cabin",
        "qty": 10,
        "rate": 4600,
        "amount": 46000,
        "start_date": "2023-11-01",
        "stop_date": "2024-10-17",
    },
    {
        "idx": 8,
        "item_code": "Parking (2S)",
        "qty": 10,
        "rate": 1000,
        "amount": 10000,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 9,
        "item_code": "Parking (2S)",
        "qty": 10,
        "rate": 0,
        "amount": 0,
        "start_date": "2024-11-01",
        "stop_date": "2025-10-31",
    },
    {
        "idx": 10,
        "item_code": "Executive Desk (4*2)",
        "qty": "8",
        "rate": "1100",
        "amount": "8800",
        "start_date": "2024-11-15",
        "stop_date": "2025-01-31",
    },
    {
        "idx": 11,
        "item_code": "Manager Cabin",
        "qty": "3",
        "rate": "5200",
        "amount": "15600",
        "start_date": "2024-10-10",
        "stop_date": "2024-11-10",
    },
    {
        "idx": 12,
        "item_code": "Conference Table",
        "qty": 1,
        "rate": "20000",
        "amount": "20000",
        "start_date": "2024-11-05",
        "stop_date": "2024-11-20",
    },
    {
        "idx": 13,
        "item_code": "Parking (2S)",
        "qty": 5,
        "rate": "1000",
        "amount": "5000",
        "start_date": "2024-11-15",
        "stop_date": "2025-02-28",
    },
    {
        "idx": 14,
        "item_code": "Reception Desk",
        "qty": 2,
        "rate": "7000",
        "amount": "14000",
        "start_date": "2024-11-01",
        "stop_date": "2025-03-31",
    },
    {
        "idx": 15,
        "item_code": "Reception Desk",
        "qty": 1,
        "rate": "7000",
        "amount": "7000",
        "start_date": "2024-11-10",
        "stop_date": "2024-11-25",
    },
    {
        "idx": 16,
        "item_code": "Breakout Area",
        "qty": 3,
        "rate": "3000",
        "amount": "9000",
        "start_date": "2024-01-01",
        "stop_date": "2024-01-31",
    }
]

In [10]:
def parse_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d").date()

def get_month_start_end(target_month):
    year, month = map(int, target_month.split("-"))
    start = datetime(year, month, 1).date()
    _, last_day = calendar.monthrange(year, month)
    end = datetime(year, month, last_day).date()
    return start, end

def get_active_days(start_date, stop_date, month_start, month_end):
    start = parse_date(start_date) if isinstance(start_date, str) else start_date
    stop = parse_date(stop_date) if isinstance(stop_date, str) else stop_date
    effective_start = max(start, month_start)
    effective_stop = min(stop, month_end)
    if effective_stop < effective_start:
        return 0
    return (effective_stop - effective_start).days + 1

def get_total_days_in_month(target_month):
    year, month = map(int, target_month.split("-"))
    _, last_day = calendar.monthrange(year, month)
    return last_day

In [16]:
def generate_monthly_bill(item_list: list, target_month: str) -> dict:

    month_start, month_end = get_month_start_end(target_month)
    total_days = get_total_days_in_month(target_month)
    
    grouped_items = {}
    
    for item in item_list:
        try:
            rate = float(item["rate"])
            qty = int(item["qty"])
        except (ValueError, TypeError):
            continue
        
        start_date = item["start_date"]
        stop_date = item["stop_date"]
        
        active_days = get_active_days(start_date, stop_date, month_start, month_end)
        if active_days == 0:
            continue  
        
        amount = (rate * qty * active_days) / total_days
        
        billing_period = f"{max(parse_date(start_date), month_start).strftime('%Y-%m-%d')} to {min(parse_date(stop_date), month_end).strftime('%Y-%m-%d')}"
        group_key = (item["item_code"], rate, billing_period)
        
        if group_key not in grouped_items:
            grouped_items[group_key] = {
                "item_code": item["item_code"],
                "rate": rate,
                "qty": 0,
                "amount": 0.0,
                "billing_period": billing_period
            }
        
        grouped_items[group_key]["qty"] += qty
        grouped_items[group_key]["amount"] += amount
    
    line_items = [
        {
            "item_code": item["item_code"],
            "rate": item["rate"],
            "qty": item["qty"],
            "amount": round(item["amount"], 2),
            "billing_period": item["billing_period"]
        }
        for item in grouped_items.values()
    ]
    
    total_revenue = sum(item["amount"] for item in line_items)
    
    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }

In [18]:
result = generate_monthly_bill(item_list, "2024-11")
print(json.dumps(result, indent=2))

{
  "line_items": [
    {
      "item_code": "Executive Desk (4*2)",
      "rate": 1080.0,
      "qty": 25,
      "amount": 27000.0,
      "billing_period": "2024-11-01 to 2024-11-30"
    },
    {
      "item_code": "Executive Desk (4*2)",
      "rate": 1000.0,
      "qty": 5,
      "amount": 5000.0,
      "billing_period": "2024-11-01 to 2024-11-30"
    },
    {
      "item_code": "Manager Cabin",
      "rate": 5000.0,
      "qty": 5,
      "amount": 25000.0,
      "billing_period": "2024-11-01 to 2024-11-30"
    },
    {
      "item_code": "Parking (2S)",
      "rate": 1000.0,
      "qty": 10,
      "amount": 10000.0,
      "billing_period": "2024-11-01 to 2024-11-30"
    },
    {
      "item_code": "Parking (2S)",
      "rate": 0.0,
      "qty": 10,
      "amount": 0.0,
      "billing_period": "2024-11-01 to 2024-11-30"
    },
    {
      "item_code": "Executive Desk (4*2)",
      "rate": 1100.0,
      "qty": 8,
      "amount": 4693.33,
      "billing_period": "2024-11-15 to 2024-11